In [987]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

import flask
from flask_cors import CORS

import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame

import os
from os import environ
import requests
import io

import datetime
from dateutil.relativedelta import relativedelta

import socket

import seaborn as sns
from itertools import groupby

In [988]:
Data=pd.read_csv('SQLsnippet_OPO.csv')
Data.columns=[x.replace(' ','') for x in Data.columns]
#Data=Data.loc[~pd.isnull(Data['outbound_seats']),:]
observation_date=sorted(Data['observation_date'].unique())
Data=Data.loc[Data['observation_date']==observation_date[len(observation_date)-1],:]

In [989]:
Data_lg_oneway=Data.loc[(Data['carrier']=='LG')&(Data['is_one_way']==1),
['origin','destination','min_stay','outbound_departure_time','outbound_departure_date','price_exc']]
Data_lg_oneway=Data_lg_oneway.rename(columns={'price_exc': 'price_outbound'})
Data_lg_oneway['min_stay']='one way'
Data_lg_oneway=Data_lg_oneway\
.groupby(['origin','destination','min_stay','outbound_departure_time','outbound_departure_date'])['price_outbound'].min().reset_index()

Data_lg_twoway=Data.loc[(Data['carrier']=='LG')&(Data['is_one_way']==0),
['origin','destination','min_stay','outbound_departure_time','inbound_departure_time','outbound_departure_date','price_outbound','price_inbound']]
Data_lg_twoway=Data_lg_twoway\
.groupby(['origin','destination','min_stay','outbound_departure_time','inbound_departure_time','outbound_departure_date'])[['price_outbound','price_inbound']].min().reset_index()

In [990]:
Data_comp_oneway=Data.loc[(Data['carrier']!='LG')&(Data['is_one_way']==1),
['carrier','origin','destination','min_stay','outbound_departure_time','outbound_departure_date','price_exc','price_outbound']]
Data_comp_oneway['price_exc']=Data_comp_oneway[['price_exc','price_outbound']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0],axis=1)
Data_comp_oneway=Data_comp_oneway[[x for x in Data_comp_oneway.columns if x!='price_outbound']]
Data_comp_oneway=Data_comp_oneway.rename(columns={'price_exc': 'price_outbound'})
Data_comp_oneway['min_stay']='one way'
Data_comp_oneway=Data_comp_oneway\
.groupby(['carrier','origin','destination','min_stay','outbound_departure_time','outbound_departure_date'])['price_outbound'].min().reset_index()

Data_comp_twoway=Data.loc[(Data['carrier']!='LG')&(Data['is_one_way']==0),
['carrier','origin','destination','min_stay','outbound_departure_time','inbound_departure_time','outbound_departure_date','price_outbound','price_inbound']]
Data_comp_twoway=Data_comp_twoway\
.groupby(['carrier','origin','destination','min_stay','outbound_departure_time','inbound_departure_time','outbound_departure_date'])[['price_outbound','price_inbound']].min().reset_index()

In [991]:
Data_oneway=Data_lg_oneway.merge(Data_comp_oneway,on=['origin','destination','min_stay','outbound_departure_date'],how='left')

Data_oneway['outbound_departure_time_x']=Data_oneway['outbound_departure_time_x']\
.apply(lambda x: datetime.datetime.strptime('1970-01-01'+' '+x,'%Y-%m-%d %H:%M'))
Data_oneway['outbound_departure_time_y']=Data_oneway['outbound_departure_time_y']\
.apply(lambda x: datetime.datetime.strptime('1970-01-01'+' '+x,'%Y-%m-%d %H:%M'))

Data_oneway['outbound_time_diff']=(Data_oneway['outbound_departure_time_x']-Data_oneway['outbound_departure_time_y']).apply(lambda x:np.abs(x.total_seconds()))

Data_oneway['outbound_time_diffmin']=Data_oneway.groupby(['carrier','origin','destination','min_stay','outbound_departure_time_x','outbound_departure_date'])['outbound_time_diff'].transform('min')

Data_oneway=Data_oneway.loc[Data_oneway['outbound_time_diff']==Data_oneway['outbound_time_diffmin'],:]
Data_oneway=Data_oneway.drop_duplicates(subset=['carrier','origin','destination','min_stay','outbound_departure_time_x','outbound_departure_date'])

Data_oneway['price_outbound_y']=Data_oneway[['price_outbound_y','outbound_time_diff']].apply(lambda x: np.nan if x[1]>3600 else x[0],axis=1)

Data_oneway=Data_oneway.rename(columns={'price_outbound_x': 'price_outbound_lg', 'price_outbound_y': 'price_outbound_comp',
                                       'outbound_departure_time_x': 'outbound_departure_time_lg'})

Data_oneway=Data_oneway[[x for x in Data_oneway.columns if '_y' not in x and '_x' not in x and '_diff' not in x]]

Data_oneway['outbound_departure_time_lg']=Data_oneway['outbound_departure_time_lg'].apply(lambda x: ':'.join(str(x).split(' ')[1].split(':')[0:2]))

In [992]:
Data_twoway=Data_lg_twoway.merge(Data_comp_twoway,on=['origin','destination','min_stay','outbound_departure_date'],how='left')

In [993]:
Data_twoway['outbound_departure_time_x']=Data_twoway['outbound_departure_time_x']\
.apply(lambda x: datetime.datetime.strptime('1970-01-01'+' '+x,'%Y-%m-%d %H:%M'))
Data_twoway['outbound_departure_time_y']=Data_twoway['outbound_departure_time_y']\
.apply(lambda x: datetime.datetime.strptime('1970-01-01'+' '+x,'%Y-%m-%d %H:%M'))
Data_twoway['inbound_departure_time_x']=Data_twoway['inbound_departure_time_x']\
.apply(lambda x: datetime.datetime.strptime('1970-01-01'+' '+x,'%Y-%m-%d %H:%M'))
Data_twoway['inbound_departure_time_y']=Data_twoway['inbound_departure_time_y']\
.apply(lambda x: datetime.datetime.strptime('1970-01-01'+' '+x,'%Y-%m-%d %H:%M'))

In [994]:
Data_twoway['outbound_time_diff']=(Data_twoway['outbound_departure_time_x']-Data_twoway['outbound_departure_time_y']).apply(lambda x:np.abs(x.total_seconds()))
Data_twoway['inbound_time_diff']=(Data_twoway['inbound_departure_time_x']-Data_twoway['inbound_departure_time_y']).apply(lambda x:np.abs(x.total_seconds()))

In [995]:
Data_twoway['time_diff']=Data_twoway['outbound_time_diff']+Data_twoway['inbound_time_diff']

In [996]:
Data_twoway['time_diffmin']=Data_twoway.groupby(['carrier','origin','destination','min_stay','outbound_departure_time_x',
'inbound_departure_time_x','outbound_departure_date'])['time_diff'].transform('min')

In [997]:
Data_twoway=Data_twoway.loc[Data_twoway['time_diff']==Data_twoway['time_diffmin'],:]
Data_twoway=Data_twoway.drop_duplicates(subset=['carrier','origin','destination','min_stay','outbound_departure_time_x',
'inbound_departure_time_x','outbound_departure_date'])

In [998]:
Data_twoway['price_outbound_y']=Data_twoway[['price_outbound_y','time_diff']].apply(lambda x: np.nan if x[1]>7200 else x[0],axis=1)
Data_twoway['price_inbound_y']=Data_twoway[['price_inbound_y','time_diff']].apply(lambda x: np.nan if x[1]>7200 else x[0],axis=1)

In [999]:
Data_twoway=Data_twoway.rename(columns={'price_outbound_x': 'price_outbound_lg', 'price_outbound_y': 'price_outbound_comp',
                                       'price_inbound_x': 'price_inbound_lg', 'price_inbound_y': 'price_inbound_comp',
                                       'outbound_departure_time_x': 'outbound_departure_time_lg',
                                       'inbound_departure_time_x': 'inbound_departure_time_lg'})

Data_twoway=Data_twoway[[x for x in Data_twoway.columns if '_y' not in x and '_x' not in x and '_diff' not in x]]

Data_twoway['outbound_departure_time_lg']=Data_twoway['outbound_departure_time_lg'].apply(lambda x: ':'.join(str(x).split(' ')[1].split(':')[0:2]))
Data_twoway['inbound_departure_time_lg']=Data_twoway['inbound_departure_time_lg'].apply(lambda x: ':'.join(str(x).split(' ')[1].split(':')[0:2]))

In [1000]:
Data_twoway['min_stay']=Data_twoway['min_stay'].apply(lambda x: str(x)+' days')

In [1001]:
Data=pd.concat([Data_oneway,Data_twoway])

C:\Users\35266\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [1002]:
Data['inbound_departure_time_lg']=Data['inbound_departure_time_lg'].fillna('none')

In [1003]:
Data['route']=Data['origin']+'-'+Data['destination']

Data=Data[['route','min_stay','outbound_departure_time_lg','inbound_departure_time_lg',
          'outbound_departure_date','price_outbound_lg','price_inbound_lg','carrier','price_outbound_comp','price_inbound_comp']]

# Dash

In [1004]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [1005]:
intermediate=Data[['route','min_stay','outbound_departure_time_lg','inbound_departure_time_lg']].drop_duplicates()
checkbox_values=intermediate\
.sort_values(by=['route','min_stay','outbound_departure_time_lg','inbound_departure_time_lg']).transpose().values.tolist()

In [1006]:
epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
    return int((dt - epoch).total_seconds())#* 1000.0

def get_marks_from_start_end(start, end):
    ''' Returns dict with one item per month
    {1440080188.1900003: '2015-08',
    '''
    result = []
    current = start
    while current <= end:
        result.append(current)
        current += relativedelta(hours=1)
    return {unix_time_millis(m):(str(m.strftime('%H:%M'))) for m in result}

def get_marks_from_start_end_2(seq):
    ''' Returns dict with one item per month
    {1440080188.1900003: '2015-08',
    '''
    return {unix_time_millis(m):(str(m.strftime('%H:%M'))) for m in seq}

In [1007]:
# all_marks_out=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2]))
# all_marks_in=get_marks_from_start_end(min(checkbox_values[3]),max(checkbox_values[3]))

# marks_lg=get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates())
# all_marks=dict(zip([x for x in all_marks.keys() if min([abs(x-y) for y in marks_lg.keys()])>5000],
# [all_marks[x] for x in all_marks.keys() if min([abs(x-y) for y in marks_lg.keys()])>5000]))
# all_marks.update(marks_lg)

#all_marks=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2]))
#all_marks.update(get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates()))
#all_marks.update(get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates()))

In [1008]:
PAGE_SIZE = 20

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
        html.Div([html.Div(style={'width': '10%', 'display': 'inline-block'}),
        html.Img(src=app.get_asset_url('LuxairGroup.jpg'), 
              style = {'backgroundColor' : '#66b3cc','display': 'inline-block', 'vertical-align': 'middle', 
                       'width': '10%', 'height': '7em'}),
        html.H4(children='Price Monitoring System',style={
            'textAlign': 'center',
            'color': 'black','height': '1em', 'display': 'inline-block', 'width': '64%'}),
            html.Img(src=app.get_asset_url('Luxair.jpg'), 
              style = {'backgroundColor' : '#66b3cc','display': 'inline-block', 'vertical-align': 'middle',
                       'width': '10%', 'height': '7em'}),
        html.Div(style={'width': '10%', 'display': 'inline-block', 'vertical-align': 'middle'})
                 ],
        style={'margin-bottom': '-4em'})
     ,
    # ROUTE DROPDOWN AND DATE SLIDER
    
        # DROPDOWN
    html.Div([html.Div(style={'width': '10%', 'margin-bottom': '1em', 'display': 'inline-block'}),
    
    html.Div([html.Div([html.Label('Route', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_route',
    options=[],
    placeholder="Select Route",                                 
    value='LUX-OPO',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '50%', 'margin-left': '20em'})
    ,
    html.Div([html.Label('Minimum Stay', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_stay',
    options=[],
    placeholder="Select duration",                                 
    value='',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '50%', 'margin-left': '20em'})
    ,
    html.Div([html.Div([html.Label('Outbound Departure Time', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_outtime',
    options=[],
    placeholder="Select departure time",                                 
    value='',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '45%', 'margin-left': '2em', 'display': 'inline-block'})
    ,
    html.Div([html.Label('Inbound Departure Time', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_intime',
    options=[],
    placeholder="Select departure time",                                 
    value='',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '45%', 'margin-left': '2em', 'display': 'inline-block'})],          
    style={'margin-bottom': '0.5em', 'width': '90%', 'margin-left': '0em'})],
    style={'width': '75%', 'margin-bottom': '1em', 'margin-top': '0','display': 'inline-block', 'background-color': 'lightgrey'}),
    
    html.Div(style={'width': '10%', 'margin-bottom': '0.5em', 'margin-top': '-3em', 'display': 'inline-block'})
             ]),
    # GRAPH CONTAINER                   
    html.Div([html.Div([dcc.Graph(
                    id='graph-outbound')],
    style={'width': '45%', 'float': 'right', 'display': 'inline-block', 'margin-top': '0.5em'}),
    html.Div([dcc.Graph(
                    id='graph-inbound')],
    style={'width': '45%', 'float': 'right', 'display': 'inline-block', 'margin-top': '0.5em'})
             ])
        ])

In [1009]:
@app.callback(
    Output("dropdown_route", "options"),
    [Input("dropdown_stay", "value"),
    Input("dropdown_outtime", "value"),
    Input("dropdown_intime", "value")]
    )
def set_checklist_route(values_stay,values_outtime,values_intime):
    subset=set([checkbox_values[0][i] for i in range(len(checkbox_values[0]))\
    if checkbox_values[1][i] in values_stay\
    and checkbox_values[2][i] in values_outtime\
    and checkbox_values[3][i] in values_intime])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

@app.callback(
    Output("dropdown_stay", "options"),
    [Input("dropdown_route", "value"),
    Input("dropdown_outtime", "value"),
    Input("dropdown_intime", "value")]
    )
def set_checklist_stay(values_route,values_outtime,values_intime):
    subset=set([checkbox_values[1][i] for i in range(len(checkbox_values[1]))\
    if checkbox_values[0][i] in values_route\
    and checkbox_values[2][i] in values_outtime\
    and checkbox_values[3][i] in values_intime])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

@app.callback(
    Output("dropdown_outtime", "options"),
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value"),
    Input("dropdown_intime", "value")]
    )
def set_checklist_outtime(values_route,values_stay,values_intime):
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[2]))\
    if checkbox_values[0][i] in values_route\
    and checkbox_values[1][i] in values_stay\
    and checkbox_values[3][i] in values_intime])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

@app.callback(
    Output("dropdown_intime", "options"),
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value"),
    Input("dropdown_outtime", "value")]
    )
def set_checklist_intime(values_route,values_stay,values_outtime):
    subset=set([checkbox_values[3][i] for i in range(len(checkbox_values[3]))\
    if checkbox_values[0][i] in values_route\
    and checkbox_values[1][i] in values_stay\
    and checkbox_values[2][i] in values_outtime])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [1010]:
if __name__ == '__main__':
    #app.run_server(debug=False,host='10.0.0.8',port=port[analyst])
    app.run_server(debug=False,host='localhost',port=4000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jun/2019 00:36:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2019 00:36:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2019 00:36:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2019 00:36:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2019 00:36:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2019 00:36:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2019 00:36:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2019 00:36:25] "GET /assets/LuxairGroup.jpg HTTP/1.1" 404 -
127.0.0.1 - - [26/Jun/2019 00:36:25] "GET /assets/Luxair.jpg HTTP/1.1" 404 -


In [259]:
@app.callback(
    Output("dropdown_stay", "options"),
    [Input("dropdown_route", "value")]
    )
def set_checklist_stay(values_route):
    
    subset=set([checkbox_values[1][i] for i in range(len(checkbox_values[1]))\
    if checkbox_values[0][i]==values_route])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [260]:
@app.callback(
    Output("dropdown_outtime", "options"),
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value")]
    )
def set_checklist_outtime(values_route,values_stay):
    
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[2]))\
    if checkbox_values[0][i]==values_route\
    and checkbox_values[1][i]==values_stay\
    and checkbox_values[4][i]=='LG'])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [261]:
@app.callback(
    Output("dropdown_intime", "options"),
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value")
    #,
    #Input("dropdown_outtime", "value")
    ]
    )
def set_checklist_intime(values_route,values_stay):
                         #,values_outtime):
    
    subset=set([checkbox_values[3][i] for i in range(len(checkbox_values[3]))\
    if checkbox_values[0][i]==values_route\
    and checkbox_values[1][i]==values_stay\
    #and checkbox_values[2][i]==values_outtime\
    and checkbox_values[4][i]=='LG'])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [262]:
@app.callback(
    [Output("slider_outtime", "min"),
    Output("slider_outtime", "max"),
    Output("slider_outtime", "marks")],
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value")]
    )
def set_checklist_depdate(values_route,values_stay):
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[2]))\
    if checkbox_values[0][i]==values_route\
    and checkbox_values[1][i]==values_stay\
    #and checkbox_values[2][i]==values_outtime\
    #and checkbox_values[4][i]=='LG'
    ])
    minimum=min([unix_time_millis(x) for x in subset])
    maximum=max([unix_time_millis(x) for x in subset])
    value=[minimum,maximum]
    marks=get_marks_from_start_end([x for x in subset if unix_time_millis(x)==minimum][0],
                                   [x for x in subset if unix_time_millis(x)==maximum][0])
    return minimum, maximum, marks

In [263]:
colors_carrier=dict(zip(Data['carrier'].unique(),['yellow','blue','red','orange']))
line_inout=dict(zip(['price_outbound_min','price_outbound_min'],['dash','dot']))

In [ ]:
@app.callback(Output('graph-outbound', 'figure'),
     #,
     # Output('graph', 'figure')
    [Input('dropdown_route', 'value'),
    Input('dropdown_stay', 'value'),
    Input('dropdown_outtime', 'value')]
    )
def update_plot(value_route, value_stay, value_outtime):
    
    dff_plot = Data.loc[(Data['route']=='value_route')\
                   &Data['value_stay']==''.apply(lambda x: unix_time_millis(x)>=values_deptime[0] and unix_time_millis(x)<=values_deptime[1])\
                   &Data['min_stay'].apply(lambda x: x in values_stay),:].sort_values(by=['outbound_departure_date'])
    carriers=dff_plot['carrier'].unique()
    
    traces = []
        
    for i in ['price_outbound_min','price_inbound_min']:
        for j in carriers:
            traces.append(go.Scatter(
               x=dff_plot.loc[dff_plot['carrier']==j,'outbound_departure_date'].tolist(),
               y=dff_plot.loc[dff_plot['carrier']==j,i].tolist(),
               mode = 'lines',
               name = j+'/'+i,
               line = dict(
               dash = line_inout[i],
               color =colors_carrier[j],
               width = 2
                 )
            ))
   
        return {'data': traces, 'layout': 
        go.Layout(title=go.layout.Title(
        text='tst',
        xref='paper',
        x=0.5),
        #xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
        #yaxis={'title': 'Bid Price', 'range': [0,dff_plot['price_exc_min'].max()+5]},
        height=600)}
    else:   
        return {'data': []}         

In [ ]:
@app.callback(Output('graph-inbound', 'figure'),
     #,
     # Output('graph', 'figure')
    [Input('dropdown_route', 'value'),
    Input('deptime_RangeSlider', 'value'),
    Input('dropdown_stay', 'value')]
    )
def update_plot(values_route, values_deptime, values_stay):
    
    dff_plot = Data.loc[Data['route'].apply(lambda x: x in values_route)\
                   &Data['outbound_departure_time'].apply(lambda x: unix_time_millis(x)>=values_deptime[0] and unix_time_millis(x)<=values_deptime[1])\
                   &Data['min_stay'].apply(lambda x: x in values_stay),:].sort_values(by=['outbound_departure_date'])
    carriers=dff_plot['carrier'].unique()
    
    traces = []
        
    for i in ['price_outbound_min','price_inbound_min']:
        for j in carriers:
            traces.append(go.Scatter(
               x=dff_plot.loc[dff_plot['carrier']==j,'outbound_departure_date'].tolist(),
               y=dff_plot.loc[dff_plot['carrier']==j,i].tolist(),
               mode = 'lines',
               name = j+'/'+i,
               line = dict(
               dash = line_inout[i],
               color =colors_carrier[j],
               width = 2
                 )
            ))
   
        return {'data': traces, 'layout': 
        go.Layout(title=go.layout.Title(
        text='tst',
        xref='paper',
        x=0.5),
        #xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
        #yaxis={'title': 'Bid Price', 'range': [0,dff_plot['price_exc_min'].max()+5]},
        height=600)}
    else:   
        return {'data': []}         

In [ ]:
if __name__ == '__main__':
    #app.run_server(debug=False,host='10.0.0.8',port=port[analyst])
    app.run_server(debug=False,host='localhost',port=4000)